# Disclaimer
The content created by Reza Fayyazi is intended for non-commercial use only. Unauthorized commercial use, reproduction, or distribution of any materials produced or associated is prohibited without explicit permission.

In [ ]:
!pip install langchain
!pip install google-generativeai langchain-google-genai
!pip install langchain_community
!pip install openai

In [13]:
import json
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from typing import List
from pydantic import BaseModel, Field
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from typing import Optional
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [5]:
class Overview(BaseModel):
    """Overview of a section of text."""
    id: str = Field(description="The CVE-ID of the vulnerability.")
    cvss: str = Field(description="The CVSS score of the vulnerability.")
    summary: str = Field(description="High-level overview of its potential impact on the affected system.")
    mitigation: str = Field(description="Provide any mitigation schemes for the vunlerability.")


class Reference(BaseModel):
    """Reference to a vulnerability."""
    references: List[str] = Field(description="References and additional URLs to the vulnerability.")

# Choose a CVE-ID for Analysis

In [6]:
#Choose a CVE-ID from Aqua Vulnerability Database: https://avd.aquasec.com/nvd/2024/
CVE_ID = "CVE-2024-0024"

## RAG using OpenAI API

In [11]:
resource_prompt = ChatPromptTemplate.from_template(
    """You are a cybersecurity expert. Consider the Relevant Information provided below and answer the Query.

Relevant Information: {context}

Query: CVE-ID: {CVE}
Given the specified CVE-ID, provide the relevant references and resources for this vulnerability.
"""
)
model_name = "gpt-4o-mini"
model = ChatOpenAI(api_key="YOUR_API_KEY", model=model_name, temperature=0.0)
output_parser = StrOutputParser()

cve_year = CVE_ID.split('-')[1]
source_url = f"https://avd.aquasec.com/nvd/{cve_year}/{CVE_ID.lower()}/"
loader = WebBaseLoader(source_url)
documents = loader.load()
doc = documents[0]
page_content = doc.page_content

reference_tagging_function = [
    convert_pydantic_to_openai_function(Reference)
]

overview_tagging_function = [
    convert_pydantic_to_openai_function(Overview)
]

tagging_model = model.bind(
    functions=reference_tagging_function,
    function_call={"name":"Reference"}
)
tagging_chain = resource_prompt | tagging_model | JsonOutputFunctionsParser()
res = tagging_chain.invoke({"CVE": CVE_ID, "context": page_content})
print(res)


main_prompt = ChatPromptTemplate.from_template(
    """You are a cybersecurity expert. Consider the Relevant Information provided below and answer the Query.

        Relevant Information: {context}

        Query: CVE-ID: {CVE}
        Given the specified CVE-ID, please provide detailed answers to the following questions:

        1. How can an attacker exploit this vulnerability? Provide a step-by-step description.

        2. What are the recommended mitigation strategies for this vulnerability?
"""
)

if isinstance(source_url, str):
    source_url = [source_url]
if not isinstance(res['references'], list):
    res['references'] = [res['references']]

# Combine the lists
all_urls = source_url + res['references']

loader = WebBaseLoader(all_urls)
docs = loader.load()

tagging_model = model.bind(
    functions=overview_tagging_function,
    function_call={"name":"Overview"}
)
tagging_chain = main_prompt | tagging_model | JsonOutputFunctionsParser()
main_res = tagging_chain.invoke({"CVE": CVE_ID, "context": docs})
print(main_res)
with open(f'./analysis_{CVE_ID}_{model_name}.json', 'w') as f:
    json.dump(main_res, f, indent=4)

{'references': ['https://nvd.nist.gov/vuln/detail/CVE-2024-0024', 'https://cwe.mitre.org/data/definitions/269.html', 'https://android.googlesource.com/platform/frameworks/base/+/6a9250ec7fc9801a883cedd7860076f42fb518ac', 'https://source.android.com/security/bulletin/2024-05-01']}
{'id': 'CVE-2024-0024', 'cvss': 'N/A', 'summary': 'CVE-2024-0024 is a vulnerability in the UserManagerService.java that allows for improper privilege management, potentially leading to local escalation of privilege. The vulnerability arises from a failure to persist or enforce user restrictions due to improper input validation, requiring user interaction for exploitation.', 'mitigation': 'To mitigate this vulnerability, it is recommended to implement proper input validation, ensure that user restrictions are enforced correctly, and follow the principle of least privilege when assigning access rights.'}


## RAG using Gemini API

In [28]:
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)

In [39]:
resource_prompt = ChatPromptTemplate.from_template(
    """You are a cybersecurity expert. Consider the Relevant Information provided below and answer the Query.

Relevant Information: {context}

Query: CVE-ID: {CVE}
Given the specified CVE-ID, provide the relevant references and resources for this vulnerability.
"""
)
model_name = "gemini-1.5-pro"
model = ChatGoogleGenerativeAI(model=model_name, google_api_key="YOUR_API_KEY", safety_settings={
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    },)
output_parser = StrOutputParser()

cve_year = CVE_ID.split('-')[1]
source_url = f"https://avd.aquasec.com/nvd/{cve_year}/{CVE_ID.lower()}/"
loader = WebBaseLoader(source_url)
documents = loader.load()
doc = documents[0]
page_content = doc.page_content

reference_tagging_function = [
    convert_pydantic_to_openai_function(Reference)
]

overview_tagging_function = [
    convert_pydantic_to_openai_function(Overview)
]

tagging_model = model.bind(
    functions=reference_tagging_function,
    #function_call={"name":"Reference"}
)
tagging_chain = resource_prompt | tagging_model | JsonOutputFunctionsParser()
res = tagging_chain.invoke({"CVE": CVE_ID, "context": page_content})
print(res)


main_prompt = ChatPromptTemplate.from_template(
    """You are a cybersecurity expert. Consider the Relevant Information provided below and answer the Query.

        Relevant Information: {context}

        Query: CVE-ID: {CVE}
        Given the specified CVE-ID, please provide detailed answers to the following questions:

        1. How can an attacker exploit this vulnerability? Provide a step-by-step description.

        2. What are the recommended mitigation strategies for this vulnerability?
"""
)

if isinstance(source_url, str):
    source_url = [source_url]
if not isinstance(res['references'], list):
    res['references'] = [res['references']]

# Combine the lists
all_urls = source_url + res['references']

loader = WebBaseLoader(all_urls)
docs = loader.load()

tagging_model = model.bind(
    functions=overview_tagging_function,
    #function_call={"name":"Overview"}
)
tagging_chain = main_prompt | tagging_model | JsonOutputFunctionsParser()
main_res = tagging_chain.invoke({"CVE": CVE_ID, "context": docs})
print(main_res)

{'references': ['https://nvd.nist.gov/vuln/detail/CVE-2024-0024', 'https://cwe.mitre.org/data/definitions/269.html', 'https://android.googlesource.com/platform/frameworks/base/+/6a9250ec7fc9801a883cedd7860076f42fb518ac', 'https://source.android.com/security/bulletin/2024-05-01']}
{'cvss': 'N/A', 'mitigation': 'Carefully manage the setting, management, and handling of privileges.  Explicitly manage trust zones in the software. Follow the principle of least privilege when assigning access rights. Consider following the principle of separation of privilege. Require multiple conditions to be met before permitting access to a system resource.', 'summary': 'In multiple methods of UserManagerService.java, there is a possible failure to persist or enforce user restrictions due to improper input validation. This could lead to local escalation of privilege with no additional execution privileges needed. User interaction is needed for exploitation.', 'id': 'CVE-2024-0024'}
